## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Step 2: Compute the camera calibration using chessboard images

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space (real undistorted chessboard corners)
               # These are the known object coordinates of the chessboard corners
               # x, y, and z from the top left corner (0,0,0) to (x, y, 0) --> flat 
imgpoints = [] # 2d points in image plane. (coordinates of the corners)

# Make a list of calibration images
images = glob.glob('1_camera_cal/calibration*.jpg')

# Array to store valid_imgages
valid_imgs = []

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        valid_imgs.append(fname)
        
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
cv2.destroyAllWindows()

## Step 3: Correct for Distortion (and save undistorted images)

In [85]:
import re
undist_imgs = []
for i in range(0, len(valid_imgs)):
    img = cv2.imread(valid_imgs[i])
    img_num = ''.join(re.findall("\d",(valid_imgs[i])))
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imshow('img',undist)
    cv2.waitKey(500)
    undist_path = str('2_camera_undist'+'/undistort'+img_num+'.jpg')
    undist_imgs.append(undist_path)
    cv2.imwrite(undist_path, undist)
cv2.destroyAllWindows()

## Step 4: Perspective Transform (and save transformed images)

In [88]:
nx = 9
ny = 6
trans_paths = []
for i in range(0, len(undist_imgs)):
    img = cv2.imread(undist_imgs[i])
    img_num = ''.join(re.findall("\d",(undist_imgs[i]))) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    if ret == True:
        cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
        offset = 100
        img_size = (gray.shape[1], gray.shape[0])
        src = np.float32([corners[0], corners[nx-1],corners[-1], corners[-nx]])
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset],
                          [img_size[0]-offset, img_size[1]-offset],
                          [offset, img_size[1]-offset]])
        M = cv2.getPerspectiveTransform(src, dst)
        warped = cv2.warpPerspective(img, M, img_size)
        cv2.imshow('img',warped)
        cv2.waitKey(500)
        trans_path = str('3_camera_transform'+'/transform'+img_num+'.jpg')
        trans_paths.append(trans_path)
        cv2.imwrite(trans_path, warped)
cv2.destroyAllWindows()

## Step 5: Undistort camera images of road

In [72]:
images = glob.glob('4_test_images/*.jpg')
undist_imgs = []
for fname in images:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, img.shape[1:], None, None)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    cv2.imshow('img',undist)
    cv2.waitKey(500)
    undist_path = str('5_test_images_undist' + '/' + img_name +'_undist.jpg')
    undist_imgs.append(undist_path)
    cv2.imwrite(undist_path, undist)
cv2.destroyAllWindows()

## Step 6a: Applying Sobel X Threshold

In [89]:
def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    # Return the result
    return binary_output

sobelx_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    grad_binary = 255*abs_sobel_thresh(img, orient = 'x', thresh_min = 20, thresh_max = 100)
    #plt.imshow(grad_binary, cmap='gray')
    cv2.imshow('img',grad_binary)
    cv2.waitKey(500)
    sobelx_path = str('6A_test_images_sobelx' + '/' + img_name +'_sobelx.jpg')
    sobelx_imgs.append(sobelx_path)
    cv2.imwrite(sobelx_path, grad_binary)
cv2.destroyAllWindows()

## Step 6B: Applying sobel x and y magnitude threshold

In [90]:
def mag_thresh(img, sobel_kernel=3, thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply x and y gradient
    sobelx = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, sobel_kernel))
    sobely = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, sobel_kernel))
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    # Return the result
    return binary_output

sobelxy_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    grad_binary = 255 * mag_thresh(img, sobel_kernel=3, thresh_min = 30, thresh_max = 100)
    #plt.imshow(grad_binary, cmap='gray')
    cv2.imshow('img',grad_binary)
    cv2.waitKey(500)
    sobelxy_path = str('6B_test_images_sobelxy' + '/' + img_name +'_sobelxy.jpg')
    sobelxy_imgs.append(sobelxy_path)
    cv2.imwrite(sobelxy_path, grad_binary)
cv2.destroyAllWindows()

## Step 6C: Direction of Gradient Threshold

In [93]:
# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    # Return the binary image
    return binary_output

gradient_imgs = []
for fname in undist_imgs:
    img = cv2.imread(fname)
    img_name = ''.join(re.findall("(?<=/)\w+",fname)) # Find .jpg name after a '/'
    grad_binary = 255 * dir_threshold(img, sobel_kernel=15, thresh = (0.7,1.3))
    cv2.imshow('img',grad_binary)
    cv2.waitKey(500)
    gradient_path = str('6C_test_images_gradient' + '/' + img_name +'_gradient.jpg')
    gradient_imgs.append(gradient_path)
    cv2.imwrite(gradient_path, grad_binary)
cv2.destroyAllWindows()

# And so on and so forth...

cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None) # returns ret, 
#mtx (camera matrix we need to transform 3D obj points to 2D image points)
#dist (distortion coefficients)
#rvecs (rotation vectors)
#tvecs (translation vectors)

dst = cv2.undistort(img, mtx, dist, None, mtx)